In [34]:
%pip install numpy
%pip install pandas
%pip install scikit-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 24.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 37.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [36]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.pipeline import Pipeline
from sklearn.impute import IterativeImputer
import numpy as np #linear algebra
import pandas as pd # data processing, CSV file read

# Input data files are available in read-only from train.csv
train = pd.read_csv('train.csv')
train = train.drop('Id', axis=1)
test = pd.read_csv('test.csv')
test_ids = test.pop('Id')

In [37]:
# Fix some NAs
train.fillna({'LotFrontage': 0}, inplace=True) # no amount of street connected to the property
train.fillna({'MiscVal': 0}, inplace=True) # no miscellaneous features

# Numerical features
train['year_qual'] = train['YearBuilt'] * train['OverallQual'] # overall condition
train['year_r_qual'] = train['YearRemodAdd'] * train['OverallQual'] # quality x remodel
train['qual_bsmt'] = train['OverallQual'] * train['TotalBsmtSF'] # quality x basement
train['qual_fl'] = train['OverallQual'] * train['1stFlrSF'] # quality x first floor
train['qual_gr'] = train['OverallQual'] * train['GrLivArea'] # quality x ground floor
train['qual_gar'] = train['OverallQual'] * train['GarageArea'] # quality x garage
train['qual_bath'] = train['OverallQual'] * train['FullBath'] # quality x bathrooms
train['qual_bed'] = train['OverallQual'] * train['BedroomAbvGr'] # quality x bedrooms
train['qual_kit'] = train['OverallQual'] * train['KitchenAbvGr'] # quality x kitchens
train['qual_fire'] = train['OverallQual'] * train['Fireplaces'] # quality x fireplaces
train['qual_gar'] = train['OverallQual'] * train['GarageCars'] # quality x garage cars
train['qual_wd'] = train['OverallQual'] * train['WoodDeckSF'] # quality x wood deck
train['qual_op'] = train['OverallQual'] * train['OpenPorchSF'] # quality x open porch
train['qual_en'] = train['OverallQual'] * train['EnclosedPorch'] # quality x enclosed porch
train['qual_3s'] = train['OverallQual'] * train['3SsnPorch'] # quality x 3 season porch
train['qual_scr'] = train['OverallQual'] * train['ScreenPorch'] # quality x screen porch
train['qual_pool'] = train['OverallQual'] * train['PoolArea'] # quality x pool area
train['qual_ms'] = train['OverallQual'] * train['MiscVal'] # quality x miscellaneous value # Not needed
train['qual_mo'] = train['OverallQual'] * train['MoSold'] # quality x month sold
train['qual_yr'] = train['OverallQual'] * train['YrSold'] # quality x year sold

numerical_features = train.select_dtypes(include=[np.number])

# Calculate the correlation matrix
correlation_matrix = numerical_features.corr()
# Set a threshold for high correlation
threshold = 0.8
# Extract highly correlated features
high_corr_features = correlation_matrix.index[abs(correlation_matrix["SalePrice"]) > threshold].tolist()
# Combine these features into a new DataFrame
high_corr_data = train[high_corr_features]
print(high_corr_data.head())



   SalePrice  qual_gr  qual_gar
0     208500    11970        14
1     181500     7572        12
2     223500    12502        14
3     140000    12019        21
4     250000    17584        24


In [38]:
trainF = train.pop('SalePrice')
combinedSet = pd.concat([train, test], axis=0, ignore_index=True)
#categoricalCols = combinedSet.select_dtypes(include=['object']).columns.tolist()
numericalCols = combinedSet.select_dtypes(include=['number']).columns.tolist()
OTS = len(train)
trainF = combinedSet.iloc[:OTS]
test = combinedSet.iloc[OTS:]

In [39]:
numerical_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(random_state=0))
])